## Save model

In [ ]:
from modules.lstm_encoder_decoder import LSTMEncoderDecoder
import numpy as np
import json

In [ ]:
encoder_decoder = LSTMEncoderDecoder(
    source_path='../input/features/lstm/3_steps/',
    weights_path='../input/convnet_weights/weights.04-0.36-87.85.hdf5',
)

In [ ]:
model_json = encoder_decoder.model.to_json()
with open('../input/model/car-identification.json', 'w') as outfile:
    json.dump(model_json, outfile)
encoder_decoder.model.save_weights('../input/model/car-identification.h5')

In [ ]:
indices = np.load('../input/features/lstm/3_steps/indices_dev.npy')
true_classes = np.load('../input/features/res_net/classes_dev_make-model.npy')[indices]
true_classnames = np.load('../input/features/res_net/classnames_dev_make-model.npy')[indices]
classes, indices = np.unique(true_classes, return_index=True)
classnames = true_classnames[indices]
np.save('../input/model/classnames.npy', classnames)

## Get images for preview

In [ ]:
import os
from shutil import rmtree, copyfile

In [ ]:
def get_dataset(name, create_dirs=True):
    names_source = f'../input/datasets/{name}_dataset/'
    files_source = f'../input/car_data/'
    target = f'../input/preview'
    
    rmtree(target)
    
    dirs = os.listdir(names_source)
    try:
        dirs.remove('.DS_Store')
    except:
        x = 1
    
    if os.path.exists(target) == False:
        os.mkdir(target)
    
    for d in dirs:
        files = os.listdir(os.path.join(names_source, d))
        if create_dirs == True:
            os.mkdir(os.path.join(target, d))
        
        for f in files:
            if create_dirs == True:
                copyfile(os.path.join(files_source, d, f), os.path.join(target, d, f))
            else:
                copyfile(os.path.join(files_source, d, f), os.path.join(target, f))

In [ ]:
get_dataset('dev')

## Save classnames

In [ ]:
import numpy as np
import pickle

In [ ]:
indices = np.load('../input/features/lstm/3_steps/indices_dev.npy')
classnames = np.load('../input/features/res_net/classnames_dev_make-model.npy')[indices]
classnames = np.unique(true_classnames)

In [ ]:
classnames_map = {}
for classname in classnames:
    classnames_map[classname] = classname.replace('_', ' ')
    
classnames_map['BMW 3_Series_2'] = 'BMW 3 Series 5th gen'
classnames_map['BMW 3_Series_3'] = 'BMW 3 Series 6th gen'
classnames_map['Bentley Continental_1'] = 'Bentley Continental 1st gen'
classnames_map['Bentley Continental_2'] = 'Bentley Continental 2nd gen'
classnames_map['Chevrolet Express_1'] = 'Chevrolet Express'
classnames_map['Chevrolet Express_2'] = 'Chevrolet Express Appearance Package'

In [ ]:
with open('../input/preview/classnames_map.json', 'wb') as fp:
    pickle.dump(classnames_map, fp, protocol=pickle.HIGHEST_PROTOCOL)